In [ ]:
import os
import sys

In [ ]:
sys.path.insert(0, '/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from common import polynom_factory

from scipy import linalg

%matplotlib inline
np.set_printoptions(precision=3, linewidth=200)

In [ ]:
order = 2
interpolant = np.eye(order+1)

In [ ]:
lgg = polynom_factory.local_gradgrad_matrix(dim=2, distribution='globatto', order=order)
lgf = polynom_factory.local_gradfunc_matrix(dim=2, distribution='globatto', order=order)

In [ ]:
dof_l_f = list(range((order+1)**2 - (order+1),(order+1)**2))
F_lr = np.dot(interpolant, lgf[1][2][((order+1)**2 - (order+1)):])


dof_r_f = list(range(order+1))
F_rl = np.dot(interpolant, lgf[1][0][:(order+1)])

In [ ]:
dist_matrix_r = np.zeros((order+1, (order+1)**2))
for num, dof in enumerate(dof_r_f):
    dist_matrix_r[num, dof] = 1.

In [ ]:
dist_matrix_l = np.zeros((order+1, (order+1)**2))
for num, dof in enumerate(dof_l_f):
    dist_matrix_l[num, dof] = 1.

In [ ]:
upper = np.hstack((lgg[0], np.dot(dist_matrix_l.T, F_rl)))
bottom = np.hstack((np.dot(dist_matrix_r.T, F_lr), lgg[0]))

In [ ]:
whole = np.vstack((upper, bottom))

In [ ]:
whole_dist = np.zeros((2*(order+1)**2,2*(order+1)**2 - (order+1)))
whole_dist_avg = np.zeros((2*(order+1)**2,2*(order+1)**2 - (order+1)))
whole_dist_map = np.zeros((2*(order+1)**2,2*(order+1)**2))
sum = 0
for i in range((order+1)**2):
    whole_dist[i,i] = 1
    if(i>=((order+1)**2 - (order+1))):
        whole_dist_avg[i,i] = .5
        whole_dist_map[i,i + 1 + order] = .5
        whole_dist_map[i,i] = .5
    else:
        whole_dist_avg[i,i] = 1
        whole_dist_map[i,i] = 1
    sum += 1
for i in range((order+1)**2):
    whole_dist[sum + i, sum+i-(order+1)] = 1
    if(i<(order+1)):
        whole_dist_avg[sum + i, sum+i-(order+1)] = .5
        whole_dist_map[sum + i, sum+i-(order+1)] = .5
        whole_dist_map[sum+i,sum+i] = .5
    else:
        whole_dist_avg[sum + i, sum+i-(order+1)] = 1
        whole_dist_map[sum+i,sum+i] = 1

In [ ]:
whole_aux = np.dot(whole_dist_map, whole)

In [ ]:
whole_classic = linalg.block_diag(lgg[0], lgg[0])

In [ ]:
whole_classic_glob = np.dot(whole_dist.T, np.dot(whole_classic, whole_dist))
whole_hype_glob = np.dot(whole_dist.T, np.dot(whole, whole_dist))

whole_glob = np.dot(whole_dist.T, np.dot(whole, whole_dist))

In [ ]:
def partial_diff_test(pointnum, point, matrix, pointdict):
    x_der = 0
    xx_der = 0
    yy_der = 0
    y_der = 0
    xy_der = 0
    
    for k,v in pointdict.items():
        dx = point[0] - k[0]
        dy = point[1] - k[1]
            
        x_der += dx*res[root,v]
        xx_der += res[root,v]*dx**2/2
        y_der += dy*res[root,v]
        yy_der += dy**2/2*res[root,v]
        xy_der += dx*dy*res[root,v]
            
    print('Mass coef: {}'.format(mass_matrix[root].sum()))
    
    print('X derivative: {}'.format(x_der))
    print('Y derivative: {}'.format(y_der))
    print('XX derivative: {}'.format(xx_der))
    print('YY derivative: {}'.format(yy_der))
    print('XY_derivative: {}'.format(xy_der), end='\n\n')

In [ ]:
from common import polynom_factory
import numpy as np
from itertools import product
sizes = [(0.,float(2**i)) for i in range(6)]

for size in sizes:
    mass_matrix = polynom_factory.local_funcfunc_matrix(order=2, dim=2, distribution='globatto', size=size)
    gradgrad = polynom_factory.local_gradgrad_matrix(order=2, distribution='globatto', dim=2, size=size)
    gradfunc = polynom_factory.local_gradfunc_matrix(order=2, distribution='globatto', dim=2, size=size)[1][0]
    print(size)
    for (point,root) in gradgrad[1].items():
        print(point)
        partial_diff_test(matrix=gradgrad[0], point=point, pointnum=root, pointdict=gradgrad[1])